# Exercise 15: Power analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below. 

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill. 

`c` is reading comprehension, and `x` is word reading experience.

In [36]:
sample_size = 100 # How many children in data set? 
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0. 
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, 
                          beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
      # WRITE YOUR CODE HERE
      # NOTE: my previous submission included a random seed setting for reproducibility.

      # Initialize a dataframe
      df <- data.frame(matrix(ncol = 3, nrow = sample_size))
      colnames(df) <- c("Age", "Experience", "Comprehension")

      # Generate the data (with rounding)
      df$Age <- round(runif(sample_size, age_lo, age_hi)) |> as.integer() # generate ages
      df$Experience <- round(beta_xa * df$Age + beta_x0 + rnorm(sample_size, sd = sd_x)) |> as.integer() # generate experiences
      df$Comprehension <- round(beta_ca * df$Age + beta_cx * df$Experience + beta_c0 + rnorm(sample_size, sd = sd_c)) |> 
            as.integer() # generate comprehension score

      return(df)
}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,Age,Experience,Comprehension
,<int>,<int>,<int>
1,99,93,408
2,138,81,289
3,181,50,304
4,138,41,209
5,128,-32,-110
6,125,88,430


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data. 

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously. 

In [37]:
# WRITE YOUR CODE HERE
library(mediation)

run_analysis <- function(data) {
    fitM <- lm(Experience ~ Age, data=dat) #Step 1: IV on M
    fitY <- lm(Comprehension ~ Experience + Age, data=dat) #Step 2: IV and M on DV
    fitMed <- mediate(fitM, fitY, treat="Age", mediator="Experience")
    # print(summary(fitMed))
    return(c(fitMed$d0, fitMed$d0.p, fitMed$z0, fitMed$z0.p))
}

results_0 <- run_analysis(dat)


print(results_0)

[1] 1.7326613 0.0000000 0.5311654 0.0100000


---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [40]:
library(tidyverse)
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)
    
    # Start simulating
    for (i in 1:num_simulations) {
      # WRITE YOUR CODE HERE
      data <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) # simulate the data using the defined function
      simouts[i,] <- run_analysis(data) # run the analysis, and add outputs to the matrix

    }
    
    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    ACME_cov = mean(simouts[,2] <= alpha) # FILL THIS IN 
    ADE_cov =  mean(simouts[,4] <= alpha) # FILL THIS IN

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))
}

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01. 

In [41]:
# WRITE YOUR CODE HERE

repeat_analysis(10, 0.01, sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)


$ACME_cov
[1] 1

$ADE_cov
[1] 0.1

---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run. 

In [43]:
# WRITE YOUR CODE HERE

map(.x = c(50, 75, 100, 125, 150), 
    .f = ~ repeat_analysis(.x, 0.01, sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c),
    progress=TRUE)

[[1]]
[[1]]$ACME_cov
[1] 1

[[1]]$ADE_cov
[1] 0.16


[[2]]
[[2]]$ACME_cov
[1] 1

[[2]]$ADE_cov
[1] 0.16


[[3]]
[[3]]$ACME_cov
[1] 1

[[3]]$ADE_cov
[1] 0.19


[[4]]
[[4]]$ACME_cov
[1] 1

[[4]]$ADE_cov
[1] 0.168


[[5]]
[[5]]$ACME_cov
[1] 1

[[5]]$ADE_cov
[1] 0.1733333

Print your results. 

> SEE ABOVE
> 
> My computer took longer time to run the code and after a few trials I'm leaving it as-is. (I acknowledge that something may need to be tweaked in the function definition part of the code.)

## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above? 

> *Write your response here.* 
> 
> The mediation effect appeared strong enough so that a few observations should suffice to show statistical significance.
> However, the direct effect was a little difficult to determine.

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> *Write your response here.* 
>
> The standard deviations of the noises are quite large compared to the values of age (we may refer to the plot in the previous exercise), not to mention that in our simulated model the parameters/multipliers for age were small (0.5 and 0.8) compared to those for experience (3).

**DUE:** 5pm EST, April 3, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*